In [ ]:
from google.colab import drive
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import spacy
from collections import Counter
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
!pip install torchtext==0.6.0 --quiet

from torchtext.data import Field, BucketIterator


In [ ]:
drive.mount('/gdrive') # Only needed if your data is on google drive

Mounted at /gdrive


In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/train_data1.json', 'r') as file:
    data = json.load(file)

In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/test_data1_final.json', 'r') as file:
    test_data = json.load(file)

In [ ]:
import string

# Define the remove_punctuations function
def remove_punctuations(sentence):
    punctuations = list(string.punctuation)
    punctuations.append('।')
    punctuations.append('৷')
    punctuations.append('’')
    punctuations.append('‘')
    cleaned = ""
    for letter in sentence:
        if letter not in punctuations:
            cleaned += letter
    return cleaned


In [ ]:
import re

# Define a function to check if a sentence contains English words
def contains_english_words(sentence):
    # Use regular expression to check for English letters (a-z or A-Z)
    return bool(re.search(r'[a-zA-Z]', sentence))

In [ ]:
for language_pair, language_data in data.items():
  print(language_pair)

English-Bengali
English-Gujarati
English-Hindi
English-Kannada
English-Malayalam
English-Tamil
English-Telgu


In [ ]:
# Process JSON data
source_sentences_train = []
target_sentences_train = []
len_telugu = []
len_english = []

id_train = []

for language_pair, language_data in data.items():
    if(language_pair == "English-Telgu"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"].lower()
              target = entry_data["target"].lower()
              source = remove_punctuations(source)
              target = remove_punctuations(target)
              if not contains_english_words(target):
                source_sentences_train.append(source)
                target_sentences_train.append(target)
                id_train.append(entry_id)
                len_telugu.append(len(source.split(' ')))
                len_english.append(len(target.split(' ')))

Language Pair: English-Telgu
  Data Type: Train


In [ ]:
# Calculate the maximum length of telugu and English sentences
max_length_telugu = max(len_telugu)
max_length_english = max(len_english)

print(f"Maximum Length of telugu Sentence: {max_length_telugu} words")
print(f"Maximum Length of English Sentence: {max_length_english} words")

Maximum Length of telugu Sentence: 100 words
Maximum Length of English Sentence: 81 words


In [ ]:
len(target_sentences_train)

44291

In [ ]:
test_source = []
valid_id = []
len_val=[]


for language_pair, language_data in test_data.items():
    if(language_pair == "English-Telgu"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"].lower()
              source = remove_punctuations(source)
              test_source.append(source)
              valid_id.append(entry_id)
              len_val.append(len(source.split(' ')))

Language Pair: English-Telgu
  Data Type: Test


In [ ]:
len(test_source)

12831

In [ ]:
print(max(len_val))

80


In [ ]:
nlp_en = spacy.load('en_core_web_sm')
# Tokenize English sentences
tokenized_source_sentences = []
for sentence in source_sentences_train:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_source_sentences.append(tokens)

In [ ]:
tokenized_test_source = []
for sentence in test_source:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_test_source.append(tokens)

In [ ]:
!pip install nltk

In [ ]:
pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 18.7 MB/s eta 0:00:00


In [ ]:
import nltk

# Download the Gujarati tokenizer
nltk.download("indic_nltk")

from indicnlp.tokenize import indic_tokenize


# Tokenize telugu sentences
tokenized_target_sentences = []
for sentence in target_sentences_train:
    tokens = indic_tokenize.trivial_tokenize(sentence)
    tokenized_target_sentences.append(tokens)

[nltk_data] Error loading indic_nltk: Package 'indic_nltk' not found
[nltk_data]     in index


In [ ]:
from torchtext.data import Dataset, Example, Field

# Define your custom tokenizer function for English text
def tokenize_english(text):
    return text.split()

def tokenize_telugu(text):
    return text.split()

class CustomTranslationDataset(Dataset):
    def __init__(self, source_sentences, target_sentences, source_field, target_field):
        fields = [('source', source_field), ('target', target_field)]
        examples = []
        for src, tgt in zip(source_sentences, target_sentences):
            examples.append(Example.fromlist([src, tgt], fields))
        super().__init__(examples, fields)
# Create Fields for English (source) and telugu (target) text
english = Field(tokenize=tokenize_english,
                lower=True,
                init_token="<sos>",
                eos_token="<eos>")

telugu = Field(tokenize=tokenize_telugu,
              lower=True,
              init_token="<sos>",
              eos_token="<eos>")

train_dataset = CustomTranslationDataset(tokenized_source_sentences, tokenized_target_sentences, english, telugu)


# Build vocabulary for the Fields with the same max_size
english.build_vocab(train_dataset,max_size=30000, min_freq=3)
telugu.build_vocab(train_dataset,max_size=30000, min_freq=3)



In [ ]:
print(len(english.vocab), len(telugu.vocab))

13746 20431


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator = BucketIterator(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    sort_key=lambda x: len(x.source),
    device=device
)


In [ ]:
count = 0
for data in train_iterator:
  if count < 1 :
    print("Shapes", data.source.shape, data.target.shape)
    print()
    print("English - ",*data.source, " Length - ", len(data.source))
    print()
    print("telugu - ",*data.target, " Length - ", len(data.target))
    temp_eng = data.source
    temp_telugu = data.target
    count += 1


Shapes torch.Size([18, 32]) torch.Size([19, 32])

English -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([   38,     4,    17,  1038,  2448,     6,    14,    50,     6,   153,
           17,   465,     4,     6, 13033,    95,     6,    13,     4,    68,
           17,     4,    91,     6, 10241,    12,    22,   416,    11,   462,
            6,     0], device='cuda:0') tensor([3824,  279, 1255,  401,    5,    4,  230,  120,    4,   18,    9,   16,
         382,    4,  282,   10, 7523,  142,  409,   18,    9,  332, 1413,    4,
          90,   42,   48,    7, 3304,   14,    0,  201], device='cuda:0') tensor([4408,   65,   80,  121, 1081, 1812, 1898,   20,   98,  368,  481,   13,
         846, 8809,    9,   53,   25,  442,    9,    4,   11,    9,   11, 6190,
         103, 1209,  894,    4,  345,  215,    0,    9], device='cuda:0') tensor([ 9692,    72,     4,    45,   384,     5,    53,    13,  16

In [ ]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_telugu_idx = (temp_telugu).cpu().detach().numpy()

In [ ]:

print(english.vocab.__dict__.keys())
print(list(english.vocab.__dict__.values()))
e = list(english.vocab.__dict__.values())
for i in e:
  print(i)

dict_keys(['freqs', 'itos', 'unk_index', 'stoi', 'vectors'])
[Counter({'the': 44071, 'of': 28523, 'in': 18898, ' ': 15663, 'and': 15477, 'is': 14124, 'to': 13404, 'a': 10055, 'for': 5629, 'this': 5417, 'on': 5334, 'are': 5045, 'from': 4592, 'it': 4393, 'with': 4032, 'that': 3973, 'by': 3867, 'be': 3630, 'you': 3515, 'was': 3206, 'also': 2882, 'there': 2794, 'can': 2667, 'at': 2604, 'as': 2551, 'which': 2349, 'or': 2327, 'not': 2278, 'has': 2241, 'have': 2108, 'an': 1816, 'one': 1700, 'i': 1652, 'what': 1622, 'if': 1598, 'after': 1542, 'his': 1528, 'my': 1498, 'its': 1488, 'he': 1474, 'then': 1459, 'me': 1419, 'time': 1389, 'will': 1371, 'your': 1359, 'should': 1359, 'but': 1300, 'all': 1288, 'india': 1240, 'these': 1236, 'more': 1231, 'do': 1223, 'like': 1203, 'been': 1183, 'water': 1116, 'some': 1095, 'they': 1079, 'only': 1069, 'were': 1063, 'other': 1060, 'their': 1047, 'here': 992, 's': 987, 'many': 975, 'people': 960, 'when': 921, 'get': 914, 'about': 900, 'had': 889, 'such': 884,

In [ ]:

word_2_idx = dict(e[3])
idx_2_word = {}
for k,v in word_2_idx.items():
  idx_2_word[v] = k


In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    self.embedding = nn.Embedding(input_size, embedding_size)

    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  def forward(self, x):


    embedding = self.dropout(self.embedding(x))

    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size_encoder = len(english.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(13746, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


In [ ]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()


    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    self.embedding = nn.Embedding(input_size, embedding_size)


    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)


    self.fc = nn.Linear(hidden_size, output_size)


  def forward(self, x, hidden_state, cell_state):

    x = x.unsqueeze(0)


    embedding = self.dropout(self.embedding(x))

    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    predictions = self.fc(outputs)

    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

In [ ]:
input_size_decoder = len(telugu.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(telugu.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(20431, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=20431, bias=True)
)


In [ ]:

class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):

    batch_size = source.shape[1]

    target_len = target.shape[0]
    target_vocab_size = len(telugu.vocab)

    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    hidden_state, cell_state = self.Encoder_LSTM(source)

    x = target[0]

    for i in range(1, target_len):

      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    return outputs

In [ ]:

# Hyperparameters

learning_rate = 0.001
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = telugu.vocab.stoi[""]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [ ]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(13746, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(20431, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=20431, bias=True)
  )
)

In [ ]:
def translate_sentence(model, sentence, english, telugu, device, max_length=100):
    nlp_en = spacy.load('en_core_web_sm')

    if type(sentence) == str:
        tokens = [token.text for token in nlp_en(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [telugu.vocab.stoi[""]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == telugu.vocab.stoi[""]:
            break

    translated_sentence = [telugu.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

def bleu(data, model, english, telugu, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["source"]
        trg = vars(example)["target"]

        prediction = translate_sentence(model, src,english,telugu, device)
        prediction = prediction[:-1]  # remove  token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


In [ ]:
def pred_trans(model, tokens, english, telugu, device, max_length=100):
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [telugu.vocab.stoi[""]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == telugu.vocab.stoi[""]:
            break

    translated_sentence = [telugu.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

In [ ]:
import time
from tqdm import tqdm
epoch_loss = 0.0
num_epochs = 10

sentence1 = "avoid alcohol and illicit drugs"


for epoch in range(num_epochs):
  start_time = time.time()
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  translated_sentence1 = translate_sentence(model, sentence1, english, telugu,device, max_length=100)
  print(f"Translated example sentence 1: \n {translated_sentence1}")


  model.train(True)
  for batch_idx, batch in tqdm(enumerate(train_iterator), total=len(train_iterator)):
    input = batch.source.to(device)
    target = batch.target.to(device)

    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp
    optimizer.step()
    step += 1
    epoch_loss += loss.item()


  # Calculate the time taken for the epoch
  end_time = time.time()
  epoch_time = end_time - start_time
  print(f"Time taken for epoch {epoch + 1}: {epoch_time:.2f} seconds")

  print("Epoch_Loss - {}".format(loss.item()))
  print()

print(epoch_loss / len(train_iterator))


Epoch - 1 / 10
Translated example sentence 1: 
 ['వున్నాయి', 'ఆశ్రయాలు', 'పక్కటెముకలు', 'బౌద్ధులకు', 'తీసివేయాలి', 'చోప్రాకు', 'వాసులు', 'కష్టపడి', 'పూర్తికాలేదు', 'కాంస్య', 'గుర్రపు', 'గుర్రపు', 'మధుమేహం', 'కృత్యాలను', 'జయంతి', 'పాఠశాలల్లో', 'మొలక', 'మొలక', 'సాంచి', 'సాంచి', 'సాంచి', 'లాడ్జ్', 'గణాంకాలను', 'అవలంబించారు', 'బుద్ధ', 'ఫుడ్స్', 'నిలిచింది', 'కేక్', 'కేక్', 'కేక్', 'చిత్రాల', 'కేక్', 'చేరుకోవచ్చు', 'చేరుకోవచ్చు', 'రాళ్ళపై', 'రాళ్ళపై', 'పూర్తికాలేదు', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '1949లో', '19

100%|██████████| 1385/1385 [04:04<00:00,  5.67it/s]


Time taken for epoch 1: 245.49 seconds
Epoch_Loss - 4.653209209442139

Epoch - 2 / 10
Translated example sentence 1: 
 ['ఈ', 'మరియు', 'మరియు', 'మరియు', '<eos>', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<

100%|██████████| 1385/1385 [03:58<00:00,  5.81it/s]


Time taken for epoch 2: 239.38 seconds
Epoch_Loss - 4.168992519378662

Epoch - 3 / 10
Translated example sentence 1: 
 ['ఒక', 'మరియు', 'మరియు', 'మరియు', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '

...

Time taken for epoch 8: 237.51 seconds
Epoch_Loss - 1.416512370109558

Epoch - 9 / 10
Translated example sentence 1: 
 ['ప్రమాదం', 'మరియు', 'మరియు', 'మరియు', '<eos>', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad

100%|██████████| 1385/1385 [04:01<00:00,  5.75it/s]


Time taken for epoch 9: 242.14 seconds
Epoch_Loss - 4.182177543640137

Epoch - 10 / 10
Translated example sentence 1: 
 ['గోముఖాసనం', 'మరియు', 'మరియు', 'ఆరోగ్యవ౦తమైన', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pa

100%|██████████| 1385/1385 [04:06<00:00,  5.63it/s]

Time taken for epoch 10: 246.84 seconds
Epoch_Loss - 3.5414137840270996

36.668939403979785


In [ ]:
s = test_source[10]
translation = translate_sentence(model, s, english, telugu, device, max_length=128)
cleaned_tokens = [token for token in translation if token not in ['<eos>', '<pad>']]
readable_translation = ' '.join(cleaned_tokens)
print(readable_translation)

ఆమె తర్వాత కూడా మరియు మరియు ధనం కూడా


In [ ]:
telugu_pred = []

for tokens in tokenized_test_source:
  translation = pred_trans(model, tokens, english, telugu, device, max_length=100)
  cleaned_tokens = [token for token in translation if token not in ['<eos>', '<pad>','।','৷']]
  pred = ' '.join(cleaned_tokens)
  print(pred)
  telugu_pred.append(pred)


Streaming output truncated to the last 5000 lines.
ఒక సూర్యోదయానికి చేయడానికి ఒక లవంగ లేదా మూడు శాతం లేదా లేదా లేదా
తన జైలుకు తన కారణంగా రచనా రచనా మరియు మరియు మరియు మరియు మరియు మరియు
రోగిని శిశువుకు కూడా అయితే చికిత్స ఉంచాలి
తలలో నూనెను పూయటం వల్ల తీసుకోవటంతో వలన రక్తం పెరుగుతుంది
ఓలీ మరియు మరియు మరియు మరియు
ఈ యొక్క ఈ ద్వారా ఈ ద్వారా ఈ కథను కథను కథను ద్వారా ఈ
ఎప్పుడు తన తన మరియు అతని సమయంలో వారు మరియు మరియు మరియు
ఈటానగర్ ఆలయం ఒక ఆలయం
ఈ ఎంతవరకు ఈ ఈ ఈ ఈ ఈ ఈ గుణం చాలా ఎక్కువ గుణం ఉంటుంది ఇది ఈ ఈ ఈ
గాంధీజీ షా కావటానికి తన తన కుమారుడు తన తరువాత అతని బయలుదేరిన తరువాత అతని శిఖరాగ్ర తిరిగి చేయాల్సి
రెండు వైపుల రెండు నాలుగు కిలోల నాలుగు మోహింపజేస్తుంది
హెపెటైటిస్ఎ 90 సుమారు వయస్సులో
మహాబలి తన ఉద్రిక్తత మొదటిసారి ప్రతిజ్ఞ కోసం వివాదాలను కోసం కోసం కోసం కోసం కోసం
ఇక్కడ కార్యాలయంలో మరియు మరియు అనుబంధ మంది మరియు మరియు మరియు మరియు కూడా కూడా కూడా
ఇక్కడ ఒక లేదా లేదా
ఈ మూల్ ఈ ఈ రోగి రోగి ఈ అయితే ఈ అయితే ఈ ఈ
నేను నా జాబితా నుండి నుండి నుండి నుండి మధ్యలో కావాలి
ఆమె తన తన ఇళయరాజా మరియు మరియు మరియు మరియు మర

In [ ]:
import csv

# Create a list of dictionaries where each dictionary represents a row
data = [{'valid_id': valid_id[i], 'telugu_pred': telugu_pred[i]} for i in range(len(valid_id))]

# Specify the CSV file path
csv_file_path = '/gdrive/MyDrive/telugu3.csv'

# Define the column names
fields = ['valid_id', 'telugu_pred']

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(data)

print(f'Saved predictions to {csv_file_path}')

Saved predictions to /gdrive/MyDrive/telugu3.csv


In [ ]:
print(len(telugu_pred))
print(len(test_source))

12831
12831
